<a href="https://colab.research.google.com/github/GARYTJ29/matrixStructAnalysis/blob/main/Truss_Matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
try:
  from pandas import DataFrame
  f=0
except:
  f=1
def stdmat(L,E=1,A=1):
  a = A*E/L
  mat = [[0,0,0,0],
         [0,0,0,0],
         [0,0,a,-a],
         [0,0,-a,a]]
  return mat
def tramat(c1,c2):
  dy = c2[1]-c1[1]
  dx = c2[0]-c1[0]
  if dx != 0:
    if dx>0:
      t = np.arctan(dy/dx)
    else:
      t = np.pi + np.arctan(dy/dx)
  else:
    t = np.pi/2
  c = round(np.cos(t),5)
  s = round(np.sin(t),5)
  mat = [[c,0,-s,0],
         [0,c,0,-s],
         [s,0,c,0],
         [0,s,0,c]]
  return mat

In [ ]:
#np.linalg.inv()

In [ ]:
joints = [ [],
    [0,0],[8,0],[0,6],[8,6],[0,12],[8,12]
]
Members = [ [],
    [1,3],[2,4],[3,5],[4,6],[3,4],[5,6],
    #6
    [1,4],[2,3],[3,6],[4,5]
]
JLoad = [0 , 
         #y
         0, 0, 0, 0, -30, 0,
         #x
         0, 0, -10, 0, 0, -20
         ]

In [ ]:
tr = [[]]
locmat = [[]]
labelm = [{}]
E = 200
A = 5000
nj = len(joints)-1
for i in range(1,len(Members)):
  c1 = joints[Members[i][0]]
  c2 = joints[Members[i][1]]
  dy = c2[1]-c1[1]
  dx = c2[0]-c1[0]
  #transformation matrix
  tr += [tramat(c1,c2)]
  l = (dy*dy+dx*dx)**0.5
  #local stiffness matrix
  locmat += [stdmat(l,E,A)]
  #labelling Matrix
  m1 = Members[i][0]
  m2 = Members[i][1]
  labelm += [{m1:0,m2:1,(m1+nj):2,(m2+nj):3}]



In [ ]:
glomat = [[]]
for i in range(1,len(Members)):
  ttr = np.transpose(tr[i])
  mt = np.matmul(ttr,locmat[i])
  mt2 = np.matmul(mt,tr[i])
  glomat += [mt2]

In [ ]:
GLmat = np.zeros((2*nj,2*nj))
for i in range(1,len(Members)):
  for j in labelm[i]:
    for k in labelm[i]:
      GLmat[j-1][k-1] += glomat[i][labelm[i][j]][labelm[i][k]]

In [ ]:
Restr = [1,2,7,8]

In [ ]:
Restr.sort()
rst = list(map(lambda a: a-1 ,Restr))
Nrestr = []
for i in range(1,2*nj+1):
  if i not in Restr:
    Nrestr += [i]
nrst = list(map(lambda a: a-1 ,Nrestr))

In [ ]:
Ju = []
for i in Nrestr:
  Ju.append(JLoad[i])
Jr = []
for i in Restr:
  Jr.append(JLoad[i])

In [ ]:
Kuu = np.array(GLmat)
Kuu = np.delete(Kuu, rst,axis=1)
Kuu = np.delete(Kuu, rst,axis=0)
kuuinv = np.linalg.inv(Kuu)
deluu = np.dot(kuuinv,Ju)

In [ ]:
delmat = [0]*(2*nj+1)
for i in range(len(Nrestr)):
  delmat[Nrestr[i]]=deluu[i]

In [ ]:
print("\nLocal Stiffness Matrix (Global Coordintes):")
for i in range(1,len(Members)):
  print("For Member", i , ":")
  print(np.array(glomat[i]))


Local Stiffness Matrix (Global Coordintes):
For Member 1 :
[[ 166666.66666667 -166666.66666667       0.               0.        ]
 [-166666.66666667  166666.66666667       0.               0.        ]
 [      0.               0.               0.               0.        ]
 [      0.               0.               0.               0.        ]]
For Member 2 :
[[ 166666.66666667 -166666.66666667       0.               0.        ]
 [-166666.66666667  166666.66666667       0.               0.        ]
 [      0.               0.               0.               0.        ]
 [      0.               0.               0.               0.        ]]
For Member 3 :
[[ 166666.66666667 -166666.66666667       0.               0.        ]
 [-166666.66666667  166666.66666667       0.               0.        ]
 [      0.               0.               0.               0.        ]
 [      0.               0.               0.               0.        ]]
For Member 4 :
[[ 166666.66666667 -166666.66666667     

In [ ]:
print("\nComplete structure stiffness matrix:")
#import sys
if f:
  print(np.array(GLmat))
else:
  print(DataFrame(GLmat).to_string(index=False,header=False))



Complete structure stiffness matrix:
 202666.666667       0.000000 -166666.666667  -36000.000000       0.000000       0.000000  48000.0      0.0       0.0  -48000.0       0.0       0.0
      0.000000  202666.666667  -36000.000000 -166666.666667       0.000000       0.000000      0.0 -48000.0   48000.0       0.0       0.0       0.0
-166666.666667  -36000.000000  405333.333333       0.000000 -166666.666667  -36000.000000      0.0  48000.0       0.0       0.0       0.0  -48000.0
 -36000.000000 -166666.666667       0.000000  405333.333333  -36000.000000 -166666.666667 -48000.0      0.0       0.0       0.0   48000.0       0.0
      0.000000       0.000000 -166666.666667  -36000.000000  202666.666667       0.000000      0.0      0.0       0.0   48000.0  -48000.0       0.0
      0.000000       0.000000  -36000.000000 -166666.666667       0.000000  202666.666667      0.0      0.0  -48000.0       0.0       0.0   48000.0
  48000.000000       0.000000       0.000000  -48000.000000       0.000000

In [ ]:
print("\nUnrestrained stiffness sub-matrix:")
if f:
  print(np.array(Kuu))
else:
  print(DataFrame(Kuu).to_string(index=False,header=False))



Unrestrained stiffness sub-matrix:
 405333.333333       0.000000 -166666.666667  -36000.000000       0.0       0.0       0.0  -48000.0
      0.000000  405333.333333  -36000.000000 -166666.666667       0.0       0.0   48000.0       0.0
-166666.666667  -36000.000000  202666.666667       0.000000       0.0   48000.0  -48000.0       0.0
 -36000.000000 -166666.666667       0.000000  202666.666667  -48000.0       0.0       0.0   48000.0
      0.000000       0.000000       0.000000  -48000.000000  253000.0 -125000.0       0.0  -64000.0
      0.000000       0.000000   48000.000000       0.000000 -125000.0  253000.0  -64000.0       0.0
      0.000000   48000.000000  -48000.000000       0.000000       0.0  -64000.0  189000.0 -125000.0
 -48000.000000       0.000000       0.000000   48000.000000  -64000.0       0.0 -125000.0  189000.0


In [ ]:
print("\nInverse of unrestrained stiffness sub-matrix:")
if f:
  print(np.array(kuuinv))
else:
  print(DataFrame(kuuinv).to_string(index=False,header=False))


Inverse of unrestrained stiffness sub-matrix:
 5.548140e-06 -4.518595e-07  5.615083e-06 -3.849174e-07  0.000003  0.000002  0.000007  0.000007
-4.518595e-07  5.548140e-06 -3.849174e-07  5.615083e-06 -0.000002 -0.000003 -0.000007 -0.000007
 5.615083e-06 -3.849174e-07  1.129711e-05 -7.028926e-07  0.000003  0.000002  0.000009  0.000009
-3.849174e-07  5.615083e-06 -7.028926e-07  1.129711e-05 -0.000002 -0.000003 -0.000009 -0.000009
 2.592149e-06 -1.907851e-06  2.874793e-06 -1.625207e-06  0.000011  0.000008  0.000013  0.000013
 1.907851e-06 -2.592149e-06  1.625207e-06 -2.874793e-06  0.000008  0.000011  0.000013  0.000013
 6.690496e-06 -6.809504e-06  9.282645e-06 -8.717355e-06  0.000013  0.000013  0.000034  0.000031
 6.809504e-06 -6.690496e-06  8.717355e-06 -9.282645e-06  0.000013  0.000013  0.000031  0.000034


In [ ]:
print("\nJoint Load Vector is:")
#for x in JLoad:
#  print(x)
print(np.array(JLoad[1::]).reshape(len(JLoad)-1,1))


Joint Load Vector is:
[[  0]
 [  0]
 [  0]
 [  0]
 [-30]
 [  0]
 [  0]
 [  0]
 [-10]
 [  0]
 [  0]
 [-20]]


In [ ]:
print("\nUnrestrained displacements are:")
for i in range(3,nj+1):
  print(f"Joint {i} : δy = {delmat[i]} m  | δx =  {delmat[i+nj]} m")


Unrestrained displacements are:
Joint 3 : δy = -0.0003305640495867766 m  | δx =  -0.00045821487603305727 m
Joint 4 : δy = 0.00016443595041322283 m  | δx =  -0.0003817851239669416 m
Joint 5 : δy = -0.0005420082644628096 m  | δx =  -0.0010207995867768582 m
Joint 6 : δy = 0.00022299173553718972 m  | δx =  -0.0010767004132231392 m


In [ ]:
jvector = [0]*(2*nj+1)
print("\nForces on Each Member are (Fy,Fx are shown in global Coordinate system):")
for i in range(1,len(Members)):
  print("Member ",i,":")
  Tempdel = []
  c1 = Members[i][0]
  c2 = Members[i][1]
  for j in [c1,c2,c1+nj,c2+nj]:
    Tempdel.append(delmat[j])
  Ftemp = np.dot(glomat[i],Tempdel)
  #Ftemp = list(map(lambda a: round(a,3),Ftemp))
  print("\tDisplacement Vector is ",Tempdel)
  print("\tForce Vector is ", Ftemp)
  print(f"\tJoint {c1} : Fy = {round(Ftemp[0],3)} kN | Fx = {round(Ftemp[2],3)} kN")
  print(f"\tJoint {c2} : Fy = {round(Ftemp[1],3)} kN | Fx = {round(Ftemp[3],3)} kN")
  jvector[c1] +=Ftemp[0]
  jvector[nj + c1] +=Ftemp[2]
  jvector[c2] +=Ftemp[1]
  jvector[1*nj +c2] +=Ftemp[3]



Forces on Each Member are (Fy,Fx are shown in global Coordinate system):
Member  1 :
	Displacement Vector is  [0, -0.0003305640495867766, 0, -0.00045821487603305727]
	Force Vector is  [ 55.09400826 -55.09400826   0.           0.        ]
	Joint 1 : Fy = 55.094 kN | Fx = 0.0 kN
	Joint 3 : Fy = -55.094 kN | Fx = 0.0 kN
Member  2 :
	Displacement Vector is  [0, 0.00016443595041322283, 0, -0.0003817851239669416]
	Force Vector is  [-27.40599174  27.40599174   0.           0.        ]
	Joint 2 : Fy = -27.406 kN | Fx = 0.0 kN
	Joint 4 : Fy = 27.406 kN | Fx = 0.0 kN
Member  3 :
	Displacement Vector is  [-0.0003305640495867766, -0.0005420082644628096, -0.00045821487603305727, -0.0010207995867768582]
	Force Vector is  [ 35.24070248 -35.24070248   0.           0.        ]
	Joint 3 : Fy = 35.241 kN | Fx = 0.0 kN
	Joint 5 : Fy = -35.241 kN | Fx = 0.0 kN
Member  4 :
	Displacement Vector is  [0.00016443595041322283, 0.00022299173553718972, -0.0003817851239669416, -0.0010767004132231392]
	Force Vector

In [ ]:
Kru = np.array(GLmat)
Kru = np.delete(Kru,nrst, axis=0)
Kru = np.delete(Kru,rst, axis=1)

#print(Kru)
#kruinv = np.linalg.inv(Kru)
Fr = np.dot(Kru,deluu)-Jr
print("\nReaction Forces:")
Frar = [{} for _ in range((nj+1))]
for i in range(len(Restr)):
  if(Restr[i]<=nj):
   Frar[Restr[i]]['Fy'] = Fr[i]
  else:
    Frar[Restr[i]-nj]['Fx'] = Fr[i]
#print(Frt)

for i in range(1,nj+1):
  if Frar[i]:
    print(f"Joint {i} :",end="\n")
    if 'Fy' in Frar[i]:
      print(f"\t Fy = {round(Frar[i]['Fy'],3)} kN",end=" ")
    if 'Fy' in Frar[i] and 'Fx' in Frar[i]:
      print("|",end=" ")
    else:
      print("\t",end=" ")
    if 'Fx' in Frar[i]:
      print(f" Fx = {round(Frar[i]['Fx'],3)} kN",end=" ")
    print("")
  


Reaction Forces:
Joint 1 :
	 Fy = 67.5 kN |  Fx = 16.541 kN 
Joint 2 :
	 Fy = -37.5 kN |  Fx = 13.459 kN 


In [ ]:
print("\nJoint Forces Vector is :\n")
print(np.array(list(map(np.around,jvector[1::],[3]*(2*nj)))).reshape(2*nj,1))


Joint Load Vecor is :

[[ 67.5  ]
 [-37.5  ]
 [ -0.   ]
 [ -0.   ]
 [-30.   ]
 [  0.   ]
 [ 16.541]
 [ 13.459]
 [-10.   ]
 [  0.   ]
 [  0.   ]
 [-20.   ]]
